In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

d_loss_values = []
g_loss_values = []

# Define constants
IMG_SIZE = 128  # Set to 128x128 for the new output size
LATENT_DIM = 200  # Updated latent dimension
BATCH_SIZE = 150  # Updated batch size
EPOCHS = 300

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GaussianNoise(nn.Module):
    def __init__(self, mean=0.0, std=0.1):
        super(GaussianNoise, self).__init__()
        self.mean = mean
        self.std = std

    def forward(self, x):
        if self.training:
            noise = torch.randn_like(x) * self.std + self.mean
            return x + noise
        return x

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = IMG_SIZE // 8  # Adjusted for 128x128 images
        self.l1 = nn.Sequential(
            nn.Linear(LATENT_DIM, 128 * self.init_size ** 2),
            GaussianNoise()
        )
        self.conv_blocks = nn.Sequential(
            nn.ConvTranspose2d(100, 2048, 4, 1, 0, bias=False),
            nn.BatchNorm2d(2048),
            nn.ReLU(True),
            nn.ConvTranspose2d(2048, 1024, 4, 2, 1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            GaussianNoise(),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            GaussianNoise(),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            GaussianNoise(),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),
                    GaussianNoise(),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(512 * 8 * 8, 1),  # Adjusted for 128x128 images
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load Stanford Dogs dataset
dogs_data = datasets.ImageFolder("../../data/stanford_dogs", transform=transform)

# Create a DataLoader
dataloader = DataLoader(dogs_data, batch_size=BATCH_SIZE, shuffle=True)

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.001, betas=(0.5, 0.999))

# Training
for epoch in range(EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1, requires_grad=False).to(device)
        fake = torch.zeros(batch_size, 1, requires_grad=False).to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, LATENT_DIM).to(device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Loss for fake images with flipped labels
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save sample images
    if epoch % 2 == 0:
        save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_dogs_128.png", nrow=5, normalize=True)
        # Save the model
        torch.save(generator.state_dict(), f"saved_model_dcgan_dogs_128_{epoch}.pth")
        # store loss values for plotting at the end in the vectors
        d_loss_values.append(d_loss.item())
        g_loss_values.append(g_loss.item())

# Save model after last iter
save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_dogs_128.png", nrow=5, normalize=True)
torch.save(generator.state_dict(), f"saved_model_dcgan_dogs_128_{EPOCHS}.pth")

# plot loss values
plt.plot(np.arange(0, EPOCHS,2), d_loss_values, label='Discriminator loss')
plt.plot(np.arange(0, EPOCHS,2), g_loss_values, label='Generator loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss values')
plt.savefig('loss_values_stanford_128.png')
plt.show()
